#####  环境：阿里云 ubuntu  python27 tensorflowf1.4.0

## 4.1深度学习与深层神经网络

## 4.2 损失函数的定义
### 4.2.1 经典损失函数

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.13.1'

#### tf.clip_by_value可以把张量中数值限制在某个范围内

In [3]:
v= tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0]])

In [4]:
sess = tf.Session()
with sess.as_default():
    print(tf.clip_by_value(v,2.5,4.5).eval())
    print("-----------------------------------")
    print (tf.log(v).eval())
    

[[2.5 2.5 3. ]
 [4.  4.5 4.5]]
-----------------------------------
[[0.        0.6931472 1.0986123]
 [1.3862944 1.609438  1.7917595]]


In [6]:
v1= tf.constant([[1.0,2.0],[3.0,4.0]])
v2 = tf.constant([[5.0,6.0],[7.0,8.0]])
sess = tf.Session()
with sess.as_default():
    print ((v1*v2).eval())#按元素相乘
    print(tf.matmul(v1,v2).eval())#矩阵相乘

[[ 5. 12.]
 [21. 32.]]
[[19. 22.]
 [43. 50.]]


### 4.2.2 自定义损失函数

In [10]:
import tensorflow as tf

from numpy.random import RandomState

batch_size = 8

x = tf.placeholder(tf.float32,shape=(None,2),name="x-input")
y_ = tf.placeholder(tf.float32,shape=(None,1),name="y-input")

w1 = tf.Variable(tf.random_normal((2,1),stddev=1,seed=1))
y = tf.matmul(x,w1)

#定义回归预测多了和预测少了的损失函数
loss_less = 10
loss_more = 1
loss = tf.reduce_sum( tf.where(tf.greater(y,y_),(y-y_)*loss_more,(y_-y)*loss_less ))

learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

#通过随机数生成数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)
#设置回归正确的数值为两个输入的和加上一个随机量。
Y= [[x1+x2+rdm.rand()/10-0.05] for (x1,x2) in X]

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    #初始化变量
    sess.run(init_op)
    
    print sess.run(w1)
    
    STEPS = 5000
    for i in range(STEPS):
        #每次选取batch个样本进行训练
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        
        #通过选取的样本外训练神经网络并更新参数
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        if i%1000==0:
            print (sess.run(w1))


[[-0.8113182]
 [ 1.4845988]]
[[-0.81031823]
 [ 1.4855988 ]]
[[0.02119794]
 [2.1813974 ]]
[[0.48111677]
 [2.3831027 ]]
[[0.72828794]
 [2.3937883 ]]
[[0.9094245]
 [2.3333366]]


## 4.3 神经网络的优化算法

#### 正则化样例代码

In [11]:
weights = tf.constant([[1.0,-2.0],[-3.0,4.0]])
with tf.Session() as sess:
    print(sess.run( tf.contrib.layers.l1_regularizer(.5)(weights) ))
    print(sess.run( tf.contrib.layers.l2_regularizer(.5)(weights) ))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

5.0
7.5


#### 计算一个5层神经网络带L2正则化的损失函数的计算方法

In [ ]:
#  非完整可执行代码片段
#   ??这个代码要干啥呀！！
import tensorflow as tf 

# 获取一层神经网络边的权重，并将这个权重的L2正则化损失加入到名称为‘losses’的集合中
def get_weight(shape,lambdaa):#lambdaa!!!
    #生成一个变量
    var = tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    # 第一个参数是集合的名字
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(lambdaa)(var))
    return var

x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))

batch_size = 8

layer_dimension = [2,10,10,10,1]
n_layers = len(layer_dimension)

#这个变量维护前向传播最深的节点，开始的时候就是输入层
cur_layer = x
#当前层节点的个数
in_dimension = layer_dimension[0]

# 通过一个循环生成5层全连接神经网络结构
for i in range(1,n_layers):
    # layer_dimension[i] 为下一层节点的个数
    out_dimension = layer_dimension[i]
    # 生成当前层中的变量，并将这个变量的L2正则化损失加入计算图上的集合
    weight = get_weight([in_dimension,out_dimension],0.001)
    bias = tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    cur_layers = tf.nn.relu(tf.matmul(cur_layers,weight) + bias)
    # 进入下一层之前将下一层的节点个数更新为当前层节点的个数
    in_dimension = layer_dimension[i]
    
mse_loss = tf.reduce_mean(tf.square(y_-cur_layer))

tf.add_to_collection('losses',mse_loss)

loss = tf.add_n(tf.get_collection('losses'))
    

### 4.4.3 滑动平均模型

In [36]:
import tensorflow as tf


v1 = tf.Variable(0,dtype=tf.float32)
step = tf.Variable(0,trainable=False)

ema = tf.train.ExponentialMovingAverage(0.99,step)
maintain_averages_op = ema.apply([v1])
with tf.Session() as sess:
    #初始化所有变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 初始化后变量的值以及滑动平均后的值
    print sess.run([v1,ema.average(v1)])
    
    #更新v1的值到5
    sess.run(tf.assign(v1,5))
    sess.run(maintain_averages_op)
    print (sess.run([v1,ema.average(v1)]))
    
    sess.run(maintain_averages_op)
    print (sess.run([v1,ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[5.0, 4.95]
